# Chebyshev or Minimax Approximation

$$ \text{minimize  } \| Ax-b \|_{\infty} $$

where $A \in \mathbb{R}^{m \times n}$ and $b \in \mathbb{R}^m$ are given.

We can recast this problem as an LP. Consider the epigraph form of the problem.

\begin{align*}
\text{minimize  } & \qquad t\\
\text{subject to  } & \qquad \| Ax-b \|_{\infty} \leq t
\end{align*}

The constraints imply that $|a_i^Tx - b_i| \leq t$ for all $1\leq i \leq m$, where $a_i^T$ is the $i^{\text{th}}$ row of $A$. Hence,

$$ -t \leq a_i^Tx - b_i \leq t \qquad \text{ for all  } 1\leq i \leq m$$

which we can write as

$$ -t\mathbf{1} \preceq Ax - b \preceq t\mathbf{1} $$

Thus, we recast the original problem as

\begin{align*}
\text{minimize  } & \qquad t\\
\text{subject to  } & \qquad -t\mathbf{1} \preceq Ax-b  \preceq t\mathbf{1}
\end{align*}

Once we solve this inequality constrained problem with variables $t$ and $x$, the optimal value of $t$ will be the optimal value of the original problem.

We will use the log barrier method to solve this LP. For increasing values of $s$, we will minimize the objective

$$ st - \displaystyle\sum_{i=1}^m \log\left((t\mathbf{1}+Ax-b)_i\right) -\displaystyle\sum_{i=1}^m \log\left((t\mathbf{1}-Ax+b)_i\right)$$

We will work with a single variable $\mathbf{w} = \begin{bmatrix} t \\ \mathbf{x} \end{bmatrix}$. We compute the gradient of the objective. The derivative with respect to $t$ is

$$ s -  \displaystyle\sum_{i=1}^m \frac{1}{(t\mathbf{1}+Ax-b)_i} - \displaystyle\sum_{i=1}^m \frac{1}{(t\mathbf{1}-Ax+b)_i}$$

and the derivative with respect to $\mathbf{x}$ is

$$ -A^T \left( \frac{1}{t\mathbf{1}+Ax-b} - \frac{1}{t\mathbf{1}-Ax+b} \right) $$

where we use $\frac{1}{t\mathbf{1}+Ax-b}$ to indicate the vector with $i^{\text{th}}$ entry $\frac{1}{t+a_i^Tx-b_i}$

In [1]:
import numpy as np
import cvxpy as cp
import plotly.graph_objects as go
import scipy

In [2]:
# generate data
m=200
n=4
A = np.concatenate([np.random.beta(2,5,(m,1)), np.random.normal(1,0.25, (m,1)), 
                    np.random.binomial(1,0.7,(m,1)), np.random.uniform(1,2,(m,1))], axis=1)
real_weights = 10*np.random.uniform(-1,1,n)
b = A@real_weights + np.random.normal(0,1,m)

In [3]:
real_weights

array([ 1.96959127,  3.21473604, -4.69300284,  3.25182315])

In [4]:
# Original Problem
z = cp.Variable(n)
objective = cp.Minimize(cp.norm(A@z-b, 'inf'))
prob = cp.Problem(objective, [])
prob.solve()

2.5292309544332685

In [5]:
z.value

array([ 1.24500625,  3.2557107 , -5.33873836,  3.55256998])

In [6]:
# Problem recast as LP
v = cp.Variable(n+1)
objective = cp.Minimize(v[0]) # first entry of v is the value t from above; it is the value of the objective function
constraints = [-v[0]-A@v[1:]+b <=0,-v[0]+A@v[1:]-b <=0 ]
prob = cp.Problem(objective, constraints)
prob.solve()

2.52923095443327

In [7]:
v.value

array([ 2.52923095,  1.24500625,  3.2557107 , -5.33873836,  3.55256998])

We will use the variable $w$ as defined above.

In [8]:
def inf_norm(x):
    return np.max(np.abs(x))
def l2_norm(x):
    return np.sqrt(np.sum(x**2))
# objective plus log barrier, where s is parameter we will increase
def f(w, s, A, b):
    return s*w[0] - np.sum(np.log(w[0]*np.ones(m)+A@w[1:]-b)) - np.sum(np.log(w[0]*np.ones(m)-A@w[1:]+b))
def grad(w, s, A, b):
    D1 = w[0]*np.ones(m)+A@w[1:]-b
    D2 = w[0]*np.ones(m)-A@w[1:]+b
    grad_t = s - np.sum(1/D1)-np.sum(1/D2)
    grad_x = -A.T@(1/D1-1/D2)
    return np.append(grad_t,grad_x)
# backtracking line search used for each centering step
def backtrack(objective, w, s, A,b, alpha, beta, grad, descent_direction):
    t0 = 1
    # since we are dealing with logs, must verify that updated point lands in domain of log
    while ((w[0]+t0*descent_direction[0])*np.ones(m)+A@(w[1:]+t0*descent_direction[1:])-b <= 0).any() or ((w[0]+t0*descent_direction[0])*np.ones(m)-A@(w[1:]+t0*descent_direction[1:])+b <= 0).any():
        t0 = beta*t0
    while objective(w+t0*descent_direction,s,A,b) > objective(w,s,A,b)+t0*alpha*grad@descent_direction:
        t0 = beta*t0
    return t0

In [9]:
def hessian(w, A, b):
    D1 = w[0]*np.ones(m)+A@w[1:]-b
    D2 = w[0]*np.ones(m)-A@w[1:]+b
    wrt_t = np.append(np.sum(1/D1**2 +1 /D2**2),-A.T@(1/D1**2 - 1 /D2**2)).reshape(n+1,1)
    tx = (1/D1**2-1/D2**2)@A
    tx = tx.reshape(1,n)
    xx = A.T@np.diag(1/D1**2+1/D2**2)@A
    
    return np.concatenate([wrt_t,np.concatenate([tx,xx])],axis=1)

In [10]:
# Gradient Descent for centering step
# this does not run quickly
tol = 1e-5
alpha = 0.01
beta = 0.5
max_iter = 1000
i = 0
w = np.append(np.max(np.abs(b))+0.1,np.zeros(n)) # feasible initial point
s = 1
mu = 5
num_inequalities = 2*m
while num_inequalities/s >= tol:
    i=0
    # centering step
    while i <= max_iter:
        g = grad(w,s,A,b)
        if l2_norm(g) < tol:
            break
        else:
            step_length = backtrack(f,w,s, A,b,alpha,beta, g, -g)
            w = w - step_length*g
        i +=1
    s = s*mu

In [11]:
w

array([ 2.54031586,  1.17896372,  3.47874303, -5.22520614,  3.33920918])

In [12]:
# Newton's Method
tol = 1e-7
alpha = 0.1
beta = 0.5
max_iter = 50
i = 0
w = np.append(np.max(np.abs(b))+0.1,np.zeros(n)) # feasible initial point
s = 1
mu = 2
num_inequalities = 2*m
while num_inequalities/s >= tol:
    i=0
    dec = np.inf
    # centering step
    while i <= max_iter:
        g = grad(w,s,A,b)
        H = hessian(w,A,b)
        newton_step = -np.linalg.inv(H)@g
        dec = -g@newton_step
        if dec < tol:
            break
        else:
            step_length = backtrack(f,w,s, A,b,alpha,beta, g, newton_step)
            w = w+step_length*newton_step
        i +=1
    s = s*mu

In [13]:
w

array([ 2.52923096,  1.24500601,  3.25571087, -5.3387383 ,  3.55256985])